In [1]:
import tabula
import pdfplumber
import pandas as pd
import nltk
from nltk import word_tokenize
import re
from gensim.parsing.preprocessing import remove_stopwords

In [2]:
from service import Service

In [3]:
pdf = pdfplumber.open(r"Service Catalogues\SQ15-000107_A (4).pdf")
pdf

### Extract all service Tables

In [4]:
def extract_all_tables(pdf):
    
    dictionary_of_tables = {} 
    for i in range(len(pdf.pages)):
        page = pdf.pages[i]
        for table in page.extract_tables():
            dictionary_of_tables['table'+str(i)] = pd.DataFrame(table[1:],columns=table[0],index=None)
    
    return dictionary_of_tables


In [5]:
dictionary_of_tables = extract_all_tables(pdf)

In [6]:
def filtering(string):
    
    string = string.replace('\n','')
    string = re.sub('[^a-zA-Z ]+', '', string)
    string = remove_stopwords(string) 
    #print(string)
    
    return string

#### Getting List of services in Service class

In [39]:
list_of_services = []
for i in list(dictionary_of_tables.keys()):
    df = dictionary_of_tables[i]
    for index,row in df.iterrows():
        list_of_services.append(Service(str(row['Service']),filtering(str(row['Description']))))
        
         

TypeError: list indices must be integers or slices, not str

In [8]:
import docx

#### Goals Document

In [9]:
doc = docx.Document(r"Service Catalogues\Government-Goals.docx")
list_of_goals = [i.text for i in doc.paragraphs]

In [10]:
list_of_goals

['Equip our people and Australian businesses with the skills necessary to deliver world-leading digital services.',
 'Adopt better ways of working that bring people together quickly and efficiently and reduce risk.',
 'Collaborate with other sectors, including small and medium-sized enterprises, community organisations and academia.',
 'Develop sustainable platforms that we can share across government.',
 'Deliver value for people and businesses by managing costs and risks.',
 'Services will be smart and adapt to the data you choose to share.',
 'Policy and services will draw on data and analytics.',
 'Advanced technologies will improve decision-making and be transparent and auditable.',
 'Earn your trust through being strong custodians of your data.',
 'Customers will be able to access all government services digitally.',
 'Customers will have seamlessly integrated services that support your needs and life events.',
 'Customers will be able to choose a secure and easy to use digital i

### Roberta Transformer

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import pandas as pd
import numpy as np
import sent2vec

In [12]:
labels = ['Contradiction','Neutral','Entailment']
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model_rob = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
def roberta_probability(sentence1,sentence2):
    
    #print(sentence1,sentence2)
    id1 = tokenizer.encode(sentence1,sentence2, return_tensors="pt")
    
    
    logits = model_rob.forward(id1, return_dict=True).logits

    p = torch.nn.functional.softmax(logits, dim=1)
    probs = p.tolist()[0]
    labels = ['Contradiction','Neutral','Entailment']
    
    result = {}
    for keys,values in zip(labels,probs):
        result[keys] = values
    
    result['Contradiction'] = result['Contradiction']*100
    result['Neutral'] = result['Neutral']*100
    result['Entailment']= result['Entailment']*100
    
    
    
    return result

In [14]:
premises = ['Jim rides a bike to school every morning','Mary made the cake','I have a dog',
            'If you are in this class, you are smart.', 'If one study hard, one will get success',
            'The President was assasinated','If you help the needy, God will reward you.']

conclusions = ['Jim can ride a bike.','The cake was was made by Mary','I have an animal',
               'Student is in this class. Student is smart.', 'He is studying hard, He will get success',
              'The president is dead', 'Giving money to a poor man has good consequences.'] 

for (i,j) in zip(premises,conclusions):
    
    result = roberta_probability(i,j)
    
    print(result)

{'Contradiction': 0.2661688718944788, 'Neutral': 1.2188411317765713, 'Entailment': 98.51499795913696}
{'Contradiction': 0.6537551525980234, 'Neutral': 1.1484806425869465, 'Entailment': 98.1977641582489}
{'Contradiction': 0.1439891755580902, 'Neutral': 1.0745935142040253, 'Entailment': 98.781418800354}
{'Contradiction': 0.2530665136873722, 'Neutral': 3.966713696718216, 'Entailment': 95.78022360801697}
{'Contradiction': 0.2513983752578497, 'Neutral': 17.1126589179039, 'Entailment': 82.63594508171082}
{'Contradiction': 0.23067670408636332, 'Neutral': 3.747386112809181, 'Entailment': 96.02193832397461}
{'Contradiction': 0.095482321921736, 'Neutral': 20.647822320461273, 'Entailment': 79.25669550895691}


### Sentence Similarity

In [15]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
#stopset = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\indva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
sentences = [i.description for i in list_of_services]

In [17]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
    
max(len(l) for l in tokenized_sent)

74

In [18]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_service = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
tagged_service

model = Doc2Vec(tagged_service, vector_size = 100, window = 6, min_count = 1, epochs = 1000, workers=4, seed = 0)

#test_doc_vector = model.infer_vector(goal_sent[0])
#model.docvecs.most_similar(positive = [test_doc_vector], topn = 25)

#### Getting 25 Most Similar service Description

goals_service_sim = {}
goals = list_of_goals
for i in range(len(goals)):
    test_doc = word_tokenize(goals[i].lower())
    test_doc_vector = model.infer_vector(test_doc)
    goals_service_sim['Goal'+str(i)] = model.docvecs.most_similar(positive = [test_doc_vector], topn = 25)

goals_service_sim

### Semantic Similarity using ROBERTA and universal sentence encoders

In [19]:
import tensorflow as tf
import tensorflow_hub as hub

In [20]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
sbert_model = hub.load(module_url)
print ("module %s loaded" % module_url)

INFO:absl:Using C:\Users\indva\AppData\Local\Temp\tfhub_modules to cache modules.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [21]:
#from sentence_transformers import SentenceTransformer
#sbert_model = SentenceTransformer('roberta-base-nli-mean-tokens')

In [22]:
sentence_embeddings = sbert_model(sentences)

In [23]:
def create_query_vecs(goals):
    
    query_vecs = []
    
    for i in goals:
        query_vecs.append(sbert_model([i])[0])
    
    return query_vecs

In [24]:
query_vecs = create_query_vecs(list_of_goals)

In [25]:
def create_similarity_dict(query_vecs,sentences):
    
    similarity_dict = {}
    index = 0
    for i in query_vecs:
        similarity_dict['Goal'+str(index)] = []
        index+=1
        for j in sentences:
            similarity_dict['Goal'+str(index-1)].append(cosine(i, sbert_model([j])[0]))

    return similarity_dict
            

In [26]:
similarity_dict = create_similarity_dict(query_vecs,sentences)

In [27]:
similarity_dict

{'Goal0': [0.25991607,
  0.24030384,
  0.096710354,
  0.089455076,
  0.060602155,
  0.14814863,
  0.18409672,
  0.19634598,
  0.064522624,
  0.15021113,
  0.057480257,
  0.0063038296,
  0.037399746,
  -0.057090554,
  0.12670529,
  0.008358691,
  0.079356655,
  0.060994007,
  0.04566265,
  0.13550541,
  0.06487283,
  0.05466594,
  0.025652884,
  -0.033789366,
  0.07070175,
  0.099146254,
  -0.017017538,
  0.06650311,
  0.029089764,
  0.075663924,
  0.15653066,
  0.06315042,
  0.216434,
  0.12969252,
  0.18784004,
  0.2003712,
  0.28780168,
  0.056662656,
  0.13052602,
  0.33283296,
  0.11230953,
  0.080270246,
  0.07267355,
  0.15456146,
  0.24864483,
  0.16313726,
  0.12523821,
  0.052393578,
  0.06145776,
  0.13894515,
  0.09265642,
  0.09661321,
  0.114860535,
  0.08285348,
  0.15639058,
  0.09242261,
  0.085292086,
  0.094328135,
  0.026752865,
  0.18438128,
  0.08236471,
  -0.024388727,
  0.14488311,
  0.17137708,
  0.18014434,
  0.041983463,
  0.202738,
  0.0483268,
  0.10431036,


In [28]:
def get_topN_similarity(similarity_dict,N):
    top_similarity = {}
    for (key,value) in similarity_dict.items():
        servkey = (sorted(range(len(value)), key=lambda i:value[i])[:-N:-1])
        valsim = sorted(value)[:-N:-1]
        top_similarity[key] = [(i,j) for (i,j) in zip(servkey,valsim)]
        
    return top_similarity

goals_service_sim = get_topN_similarity(similarity_dict,25)
goals_service_sim

{'Goal0': [(39, 0.33283296),
  (178, 0.31909427),
  (198, 0.31481802),
  (180, 0.29052478),
  (36, 0.28780168),
  (0, 0.25991607),
  (44, 0.24864483),
  (1, 0.24030384),
  (172, 0.2346795),
  (236, 0.23307623),
  (171, 0.22949693),
  (173, 0.2254942),
  (140, 0.2243152),
  (32, 0.216434),
  (176, 0.20725223),
  (226, 0.20662154),
  (175, 0.20541254),
  (66, 0.202738),
  (155, 0.20151484),
  (170, 0.20045528),
  (35, 0.2003712),
  (150, 0.1979412),
  (7, 0.19634598),
  (190, 0.1963282)],
 'Goal1': [(79, 0.19348553),
  (62, 0.16177094),
  (171, 0.16068624),
  (193, 0.15847227),
  (113, 0.15780306),
  (105, 0.15780306),
  (88, 0.15780306),
  (75, 0.15780306),
  (70, 0.15780306),
  (121, 0.15521762),
  (96, 0.15461478),
  (10, 0.15388623),
  (9, 0.14893655),
  (149, 0.14674348),
  (154, 0.14591058),
  (115, 0.14203264),
  (177, 0.13280855),
  (60, 0.13013458),
  (179, 0.1228812),
  (189, 0.12231998),
  (174, 0.1217884),
  (47, 0.12048746),
  (150, 0.119407386),
  (269, 0.116111435)],
 'Goa

#### Entailment based approach

goal_serv_dict = {}
for g in list_of_goals:
    index = list_of_goals.index(g)
    goal_serv_dict['Goals'+str(index)] = []
    for s in list_of_services:
        result = roberta_probability(s.description,g)
        goal_serv_dict['Goals'+str(index)].append(result['Entailment'])

goal_serv_dict

goals_service_sim = get_topN_similarity(goal_serv_dict,50)
goals_service_sim

## Service-Strategy Alignment

In [29]:
alignment_dict = {}
postconditions = str()

for g in list_of_goals:
    index = list_of_goals.index(g)
    service = list_of_services[goals_service_sim['Goal'+str(index)][0][0]]
    postconditions = service.description
    goal = g
    result = roberta_probability(postconditions,goal)
    alignment_dict['Goal'+str(index)] = []
    j = 1
    while(result['Entailment'] < 75 and j < 25):
        
        try:
            #print('Goal'+str(i))s
            service = list_of_services[goals_service_sim['Goal'+str(index)][j][0]]
            postconditions= postconditions + '. ' + service.description
            result = roberta_probability(list_of_goals[index],postconditions)
            alignment_dict['Goal'+str(index)].append((j,result['Entailment']))
            print(alignment_dict)
            j+=1
            
        except IndexError as error:
            print("Postconditions too long!")
            
            j+=1
            

{'Goal0': [(1, 10.153035819530487)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165)]}
{'Goal0': [(1, 10.153035

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.019266724

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363)]}
{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.7

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

...

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

{'Goal0': [(1, 10.153035819530487), (2, 20.74311673641205), (3, 37.738630175590515), (4, 36.533427238464355), (5, 39.26369547843933), (6, 34.7922146320343), (7, 36.40815019607544), (8, 33.21482837200165), (9, 25.161367654800415), (10, 31.45865797996521), (11, 28.713762760162354), (12, 57.10785984992981), (13, 35.546875), (14, 38.01926672458649), (15, 35.10203957557678), (16, 34.62501764297485), (17, 37.32013702392578), (18, 36.771345138549805), (19, 35.484763979911804), (20, 34.55322086811066), (21, 34.52487587928772), (22, 34.47587788105011), (23, 37.13122606277466)], 'Goal1': [(1, 32.050469517707825), (2, 32.90347456932068), (3, 16.18514060974121), (4, 17.32368767261505), (5, 43.368035554885864), (6, 35.33827364444733), (7, 30.56301176548004), (8, 30.40371835231781), (9, 26.773405075073242), (10, 32.6829195022583), (11, 35.367873311042786), (12, 32.90918171405792), (13, 35.931989550590515), (14, 21.21197134256363), (15, 19.033722579479218), (16, 22.835589945316315), (17, 30.000987648

In [30]:
alignment_dict

{'Goal0': [(1, 10.153035819530487),
  (2, 20.74311673641205),
  (3, 37.738630175590515),
  (4, 36.533427238464355),
  (5, 39.26369547843933),
  (6, 34.7922146320343),
  (7, 36.40815019607544),
  (8, 33.21482837200165),
  (9, 25.161367654800415),
  (10, 31.45865797996521),
  (11, 28.713762760162354),
  (12, 57.10785984992981),
  (13, 35.546875),
  (14, 38.01926672458649),
  (15, 35.10203957557678),
  (16, 34.62501764297485),
  (17, 37.32013702392578),
  (18, 36.771345138549805),
  (19, 35.484763979911804),
  (20, 34.55322086811066),
  (21, 34.52487587928772),
  (22, 34.47587788105011),
  (23, 37.13122606277466)],
 'Goal1': [(1, 32.050469517707825),
  (2, 32.90347456932068),
  (3, 16.18514060974121),
  (4, 17.32368767261505),
  (5, 43.368035554885864),
  (6, 35.33827364444733),
  (7, 30.56301176548004),
  (8, 30.40371835231781),
  (9, 26.773405075073242),
  (10, 32.6829195022583),
  (11, 35.367873311042786),
  (12, 32.90918171405792),
  (13, 35.931989550590515),
  (14, 21.21197134256363)

### Evaluation and results

In [31]:
from operator import itemgetter

In [32]:
keys = list(alignment_dict.keys())
final_alignment = {x:max(alignment_dict[x],key=itemgetter(1)) for x in keys}

In [33]:
final_alignment

{'Goal0': (12, 57.10785984992981),
 'Goal1': (5, 43.368035554885864),
 'Goal2': (2, 27.731052041053772),
 'Goal3': (10, 53.30802798271179),
 'Goal4': (11, 34.93892848491669),
 'Goal5': (2, 27.34711468219757),
 'Goal6': (5, 73.00105094909668),
 'Goal7': (21, 36.63724660873413),
 'Goal8': (20, 68.61226558685303),
 'Goal9': (18, 46.96150720119476),
 'Goal10': (21, 45.923012495040894),
 'Goal11': (10, 40.90115129947662),
 'Goal12': (22, 30.65517544746399)}

In [34]:
strategy_service_alignment = {}

index = 0

for (key,value) in goals_service_sim.items():
    
    value = final_alignment[key]
    
    probability = value[1]
    
    strategies = goals_service_sim[key][:value[0]]
    
    
    
    goal = list_of_goals[index]
    
    aligned_services = []
    postconditions = str()
    
    for (i,v) in strategies:
        
        aligned_services.append(list_of_services[i].name)
        postconditions = postconditions + '. ' + list_of_services[i].description
        
        
    strategy_service_alignment[key] = {'Goal': goal,
                                       'Number of Services': len(aligned_services),
                                       'Aligned_services': aligned_services,
                                       'Postconditions' : postconditions,
                                       'Max_probability': probability}
        
    index+=1    
    
    
    
    

In [35]:
strategy_service_alignment

{'Goal0': {'Goal': 'Equip our people and Australian businesses with the skills necessary to deliver world-leading digital services.',
  'Number of Services': 12,
  'Aligned_services': ['International \nRoaming Solutions',
   'Mail Runs (Partner \nDepartments National \nOffice Only)',
   'Technical Analysis and \nAdvice',
   'Same Day Service',
   'Contact Centre \nManagement',
   'Australian \nGovernment \nDisaster Recovery \nCommittee',
   'Staff Notices',
   'Business Continuity \nPlan Development',
   'Classified Mail (Safe \nHand)',
   'FTP File Transfer \nServices',
   'Bulk Mail Outs',
   'Courier Mail Freight'],
  'Postconditions': '. Provision international data voice roaming services staff travelling overseas work purposes. Undertake mail delivery services National Office. Technical advice identification available technology solution options reuse buy build meet business requirements liaison Share Services Centre infrastructure specialists andor vendors infrastructure requirem

In [36]:
alignment_df = pd.DataFrame(strategy_service_alignment)
alignment_df

,Goal0,Goal1,Goal2,Goal3,Goal4,Goal5,Goal6,Goal7,Goal8,Goal9,Goal10,Goal11,Goal12
Goal,Equip our people and Australian businesses wit...,Adopt better ways of working that bring people...,"Collaborate with other sectors, including smal...",Develop sustainable platforms that we can shar...,Deliver value for people and businesses by man...,Services will be smart and adapt to the data y...,Policy and services will draw on data and anal...,Advanced technologies will improve decision-ma...,Earn your trust through being strong custodian...,Customers will be able to access all governmen...,Customers will have seamlessly integrated serv...,Customers will be able to choose a secure and ...,Customers will have access to alternatives if ...
Number of Services,12,5,2,10,11,2,5,21,20,18,21,10,22
Aligned_services,"[International \nRoaming Solutions, Mail Runs ...","[Asset Disposals, Training, advice \nand suppo...","[Reporting, Research, \nand Business Analysis,...","[Training, advice \nand support, Desktop Anywh...","[Print Management \nSolutions, Advice, People ...","[Staff Notices, Reporting, Research, \nand Bus...","[Business Intelligence (BI) \nReporting Suite,...","[Assistive \nTechnologies, Mobility, Programme...","[Manage and Maintain \nHR data, Telepresence, ...","[Staff Notices, Remote Access Systems \n(RAS),...","[Electronic Alerts, Staff Notices, Library Adv...","[Identity and Access \nManagement, FEDLINK Acc...","[Modification of \nBuilding Access, Remote Acc..."
Postconditions,. Provision international data voice roaming s...,. Provide advice business areas appropriate as...,. Direct support business areas development re...,. Provide advice support platforms policies se...,. Provide print management services including ...,. Communicate information customers services i...,. Business Intelligence Reporting suite provid...,. Provision assistive technologies fulfil reco...,. Manage maintain customers HR records support...,. Communicate information customers services i...,. Provide alert service topics These negotiate...,. Enabling authorised user management access c...,. Modification building access privileges appl...
Max_probability,57.10786,43.368036,27.731052,53.308028,34.938928,27.347115,73.001051,36.637247,68.612266,46.961507,45.923012,40.901151,30.655175


In [37]:
alignment_df.to_csv(r'service_strategy_alignment.csv')

#### P & C approach

def remove_elements(pc_list,remove_list):
    
    pc_list = pc_list.remove(remove_list)
    
    return pc_list

def getpostconds(postcond,pc_list):
    
    all_pcs = [str(postcond + '. ' + i) for i in pc_list]
    
    return all_pcs

def get_prob_list(goal,list_of_services):
    
    prob_list = []
    
    for i in list_of_services:
        prob_list.append()

goal = list_of_goals[0]
goal

import itertools

def combination_lists(services,combinations):
    
    combination_list = list(itertools.combinations(services, combinations))

    return combination_list

services = combination_lists(list_of_services,4)
len(services)
